In [7]:
import sys
from pathlib import Path
# Add the project root to Python path
sys.path.append(str(Path.cwd().parent.parent))

# Now your original imports should work
from parser_engine.language_parsers.python_parser import PythonParser
from parser_engine.models.data_models import *
from parser_engine.core.repo_analyzer import RepoIndexer

repo_path = Path('/Users/qudi/Desktop/workspace/third-party/fairscale/fairscale')

In [8]:
# print module name
def print_function(func: FunctionElement):
    print(func.name)

def print_class(cls: ClassElement):
    print(cls.name)
    for func in cls.methods:
        print_function(func)

def print_module(module: ModuleElement):
    for cls in module.classes:
        print_class(cls)

In [9]:
parsed_modules = []
parser = PythonParser()
for file in repo_path.glob('**/*.py'):
    module = parser.parse_file(file)
    parsed_modules.append(module)


fairscale.version
fairscale.__init__
experimental.__init__
tooling.layer_memory_tracker
tooling.layer_memory_tracker.find_best_reset_points(activation_sizes: List[int], num_checkpoints: int) -> Tuple[int, List[int]]
tooling.layer_memory_tracker.suggest_checkpoint_location(traces: List[LayerMemoryTrace], num_checkpoints: int, num_skipped_layers: int) -> SuggestedCheckpoints
tooling.layer_memory_tracker._assert_visualisation_library_installed() -> None
tooling.layer_memory_tracker.compare_memory_traces_in_plot(memory_traces_by_job: Dict[str, List[LayerMemoryTrace]], figsize: Tuple[int, int], capture: bool) -> Optional[Any]
tooling.layer_memory_tracker.null_context() -> Iterator[None]
tooling.layer_memory_tracker.matplotlib_figure_to_image(fig: Any) -> Any
tooling.__init__
nn.auto_shard
nn.auto_shard._get_count(param_count: Dict, node_name: str) -> int
nn.auto_shard._create_shard_to_param_count(param_count: Dict, node_name_to_shard_id: Dict) -> Dict
nn.auto_shard._split_nodes(traced_graph

In [10]:
def traverse_function(func: FunctionElement, indent=0):
    # print function content
    print(' ' * indent + func.name)
    for param in func.parameters:
        print(' ' * indent + 'Parameter: ' + param)
    #get the function body
    # Use lineno and ModuleElement.body to get the body
    module = func.module
    body = module.body
    print(' ' * indent + 'Body: ' + body)



In [11]:
traverse_function(module.classes[1].methods[0])

python_parser.py:PythonParser(BaseParser):__init__(self: Any) -> Any
Parameter: self: Any
Body: """
Python-specific code parser implementation.
"""

import ast
from pathlib import Path
from typing import List, Optional, Dict, Any, Union
from dataclasses import dataclass

from .base_parser import BaseParser
from ..models.data_models import (
    ModuleElement, ClassElement, FunctionElement, DocumentationElement
)

@dataclass
class ContextInfo:
    """Helper class to track parsing context."""
    module: Optional[ModuleElement] = None
    parent_class: Optional[ClassElement] = None
    parent_function: Optional[FunctionElement] = None
    in_async_def: bool = False

class PythonParser(BaseParser):
    """Parser for Python source code files."""
    
    language = 'Python'

    def __init__(self):
        super().__init__()

    def can_parse(self, path: Path) -> bool:
        """Check if file is a Python file."""
        return path.suffix.lower() in self.get_supported_extensions()

    

In [18]:
print_module(module)

python_parser.py:ContextInfo
python_parser.py:PythonParser
python_parser.py:__init__(self: Any) -> Any : 27
python_parser.py:can_parse(self: Any, path: Path) -> bool : 31
python_parser.py:get_supported_extensions(self: Any) -> List[str] : 35
python_parser.py:parse_file(self: Any, path: Path) -> ModuleElement : 39
python_parser.py:_parse_class(self: Any, path: Path, node: ast.ClassDef) -> ClassElement : 86
python_parser.py:_parse_decorators(self: Any, node: Union[ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef]) -> List[str] : 126
python_parser.py:_get_decorator_name(self: Any, node: Union[ast.Name, ast.Attribute]) -> str : 180
python_parser.py:_parse_function(self: Any, path: Path, node: Union[ast.FunctionDef, ast.AsyncFunctionDef]) -> FunctionElement : 196
python_parser.py:_parse_imports(self: Any, node: Union[ast.Import, ast.ImportFrom]) -> List[str] : 235
python_parser.py:_parse_docstring(self: Any, node: ast.AST) -> Optional[DocumentationElement] : 247
python_parser.py:_calcula